# Segmentación código EvoPrompt

Guillermo Segura Gómez

En este notebook se va segmentar el código de [EvoPrompt](https://github.com/beeevita/EvoPrompt) para su correcto funcionamiento. Además se tratará de utilizar prompts en español para ver su funcionamiento. 

In [1]:
# Librerías
import argparse
import json
import os
from tqdm import tqdm
import numpy as np
import heapq
import random
import sys
import time
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoTokenizer,
    OPTForCausalLM,
    AutoModelForCausalLM,
    AutoModelForMaskedLM,
    LlamaForCausalLM,
    LlamaTokenizer,
)

# Templates
from data.templates import templates
from data.template_ga import templates_2

from datasets import Dataset as Dataset2
from sacrebleu.metrics import BLEU, CHRF, TER

from utils import * # Utilidades
from dataset import TextDataset # Libreria de configuración de los datos
from llm_client import * # Libreria para llamar al modelo por API
from metrics import *


### Clase Evaluator

La clase `Evaluator` es responsable de evaluar prompts y resultados en el proceso de evolución. Configura el modelo de lenguaje, prepara los datos para la evaluación y gestiona el proceso de generación de hipótesis. Sus principales componentes incluyen:

- **Inicialización (`__init__`):** Configura el dispositivo, carga plantillas, selecciona el modelo de lenguaje y el tokenizer, y configura el logger y la ruta de salida.
- **Formación de Demostraciones (`form_demons`):** Prepara muestras de datos fuente y objetivo según la tarea especificada, formatea estas muestras utilizando una plantilla y las concatena en una cadena de demostraciones.
- **Creación de Dataset (`create_dataset`):** Prepara las demostraciones y los datos con prompts para la evaluación, maneja los datos específicos del modelo y tokeniza el dataset, creando un DataLoader para el batching.
- **Método Forward (`forward`):** Un método abstracto que debe ser sobrescrito en subclases.
- **Generación de Hipótesis (`get_generations`):** Crea un dataset para la evaluación y genera hipótesis utilizando diferentes métodos según el modelo de lenguaje.

In [2]:
class Evaluator(object):
    def __init__(self, args) -> None:
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dataset = args.dataset
        template_file = "./data/template_v2.json"
        
        templates = json.load(open(template_file, "r"))
        if "alpaca" in args.language_model:
            model = "alpaca"
        elif "gpt" in args.language_model:
            model = "gpt"

        self.instruction_placeholder = templates["instruction"][model]
        dataset = self.dataset
        if args.position in ["icl", "pre"]:
            self.template = templates[args.task]["icl"][model][dataset][0]

        elif args.position == "demon":
            self.template = templates[args.task]["icl"][model][dataset][1]
        else:
            self.template = None
        # print(self.template)
        self.model_name = args.language_model.split("/")[-1]

        self.client = None
        self.llm_config = llm_init(f"./auth.yaml", args.llm_type, args.setting)

        if "gpt" in args.language_model:
            self.tokenizer = None
            self.model = None


        self.public_out_path = args.output
        if not os.path.exists(self.public_out_path):
            os.makedirs(self.public_out_path)
        self.logger = setup_log(os.path.join(self.public_out_path, f"evol.log"))
        logger = self.logger
        logger.info("=" * 50)
        logger.info(f"dev data: {args.dev_file}")
        logger.info(f"test data: {args.test_file}")
        self.args = args
        logger.info("\n\t" + "\n\t".join(f"{k} = {v}" for k, v in vars(args).items()))
        logger.info("=" * 50)
        self.logger.info(f"Instruction placeholder: {self.instruction_placeholder}")

    def form_demons(self, task, data_store_path, prompt_num):
        if task == "cls":
            data_store = read_lines(data_store_path)
            datastore_src = [line.split("\t")[0] for line in data_store]
            datastore_tgt = [
                self.verbalizers[int(line.strip().split("\t")[1])]
                for line in data_store
            ]
            demon_src, demon_tgt = extract_n_samples_per_class(
                datastore_src, datastore_tgt, prompt_num, self.dataset
            )
        elif task in ["sim", "sum"]:
            datastore_src, datastore_tgt = self.dev_src, self.dev_tgt

            indices = list(range(prompt_num))
            demon_src, demon_tgt = [datastore_src[i] for i in indices], [
                datastore_tgt[i] for i in indices
            ]
        else:
            raise ValueError("task should be sim, sum or cls")
        demonstrations = []
        for x, y in zip(demon_src, demon_tgt):
            demonstrations.append(
                format_template(
                    src=x,
                    tgt=y,
                    template=self.template,
                )
            )
        demonstrations = "\n\n".join(demonstrations)
        return demonstrations

    def create_dataset(
        self,
        data_store_path,
        test_src_sample,
        test_tgt_sample,
        tokenizer,
        verbose=True,
        src_name="",
        tgt_name="",
        model="gpt",
        batch_size=16,
        prompt_num=0,
        prompt_pre="",
        task="",
        position="pre",
        ):

        if prompt_num > 0:
            demonstrations = (
                self.form_demons(task, data_store_path, prompt_num) + "\n\n"
            )
            print(demonstrations)
            os._exit(1)
        else:
            demonstrations = ""
        data_with_prompt = []

        if model == "gpt" and "turbo" in self.args.llm_type:
            if "turbo" in self.args.llm_type:
                data_with_prompt = test_src_sample
        else:  # davinci
            for test_src_line in test_src_sample:
                prompts = []
                example = format_template(
                    src=test_src_line,
                    src_name=src_name,
                    tgt_name=tgt_name,
                    template=self.template,
                )
                instruction_part = self.instruction_placeholder.replace(
                    "<prompt>", prompt_pre
                )

                if position in ["pre", "demon"]:  # demon includes instruction + demon
                    if "alpaca" in self.args.language_model:
                        prompts.append(instruction_part + "\n\n" + example)
                    else:
                        prompts.append(
                            instruction_part + "\n" + demonstrations + example
                        )

                elif position == "icl":  # no instruction
                    example = instruction_part + "\n" + demonstrations + example
                    prompts.append(example)
                data_with_prompt.append("\n\n".join(prompts))
        if verbose and model == "gpt":
            self.logger.info("### dataset example: " + data_with_prompt[0] + "\n")
            return data_with_prompt


    def forward(self):
        raise NotImplementedError

    def get_generations(self, prompt_pre,  eval_src, ref_texts):
        args = self.args
        batch_size = args.batch_size
        dataset = self.create_dataset(
            args.dev_file,
            eval_src,
            ref_texts,
            tokenizer=self.tokenizer,
            model=self.model_name,
            batch_size=batch_size,
            prompt_num=args.prompt_num,
            prompt_pre=prompt_pre,
            task=args.task,
            position=args.position,
        )

        hypos = []
        if "gpt" in args.language_model:
            if args.task == "cls":
                hypos = llm_cls(
                    dataset=dataset,
                    client=self.client,
                    type="davinci",
                    batch_size=self.args.batch_size,
                    max_new_tokens=self.args.max_new_tokens,
                    temperature=0,
                )
            else:
                if "davinci" in args.llm_type:
                    hypos = llm_query(
                        dataset,
                        client=self.client,
                        type=args.llm_type,
                        task=True,
                        temperature=0,
                        **self.llm_config,
                    )
                    # hypos = llm_gen(dataset)
                else:
                    for data in tqdm(dataset):
                        pred = llm_query(
                            data,
                            client=self.client,
                            type=args.llm_type,
                            task=True,
                            **self.llm_config,
                        )
                        # print(pred)
                        hypos.append(pred)
    
        return hypos

### Clase SimEvaluator

La clase `SimEvaluator` es una subclase de `Evaluator` específicamente diseñada para evaluar tareas de simplificación de texto. Extiende las capacidades de la clase `Evaluator` y añade funcionalidades específicas para la tarea de simplificación:

- **Inicialización (`__init__`):** Llama al inicializador de la clase base, establece el archivo de desarrollo (`dev_file`) y carga los datos de simplificación (fuente y objetivo) para desarrollo y prueba.
- **Método Forward (`forward`):** Genera hipótesis utilizando el método `get_generations`, calcula la puntuación SARI para evaluar la calidad de las hipótesis generadas y retorna las hipótesis junto con sus puntuaciones.

En resumen, `SimEvaluator` hereda de `Evaluator` y añade funcionalidades específicas para evaluar la tarea de simplificación de texto, aprovechando los métodos y la infraestructura general proporcionada por `Evaluator`.

In [11]:
class SimEvaluator(Evaluator):
    def __init__(self, args):
        super(SimEvaluator, self).__init__(args)
        
        args.dev_file = (
            f"./data/sim/{args.dataset}/dev_{args.sample_num}.txt"
            if args.dev_file is None
            else args.dev_file
        )
        self.dev_src, self.dev_tgt, self.test_src, self.test_tgt = load_sim_data(
            args.dataset, args.seed
        )

        # self.dev_src=[
        # "En un día soleado, María decidió ir a caminar por el parque con su perro, disfrutando del buen clima y de la tranquilidad del entorno.",
        # "El presidente anunció en una conferencia de prensa que el nuevo programa económico traerá beneficios significativos a la población.",
        # "El tren de alta velocidad reduce significativamente el tiempo de viaje entre las principales ciudades del país, permitiendo a los pasajeros llegar a su destino más rápido."
        # ]

        # self.dev_tgt=[
        # ["María fue a caminar al parque con su perro en un día soleado."],
        # ["El presidente dijo que el nuevo programa económico ayudará a la población."],
        # ["El tren rápido reduce el tiempo de viaje entre ciudades."]
        # ]

        print(self.dev_src)
        print(self.dev_tgt)

    def forward(
        self, prompt_pre="",  eval_src=None, ref_texts=None, output=None
    ):
        
        print(eval_src)
        print(ref_texts)
        hypos = self.get_generations(prompt_pre, eval_src, ref_texts)
        sari_score = cal_sari(eval_src, hypos, ref_texts)
        return {"hypos": hypos, "scores": [sari_score]}


In [12]:
args = argparse.Namespace(
    dataset="asset",
    task="sim",
    test_file=None,
    batch_size=20,
    max_new_tokens=128,
    prompt_num=0,
    dev_file=None,
    output="outputs/sim/asset/gpt/all/ga/bd2_top2_para_topk_init/topk/turbo/seed5",
    language_model="gpt",
    position="pre",
    sample_num=3,
    seed=5,
    budget=2,
    popsize=2,
    evo_mode="ga",
    llm_type="turbo",
    initial="all",
    initial_mode="para_topk",
    para_mode=None,
    ckpt_pop=None,
    template="v1",
    pred_mode="logits",
    client=False,
    cache_path="data/sim/asset/seed5/prompts_gpt.json",
    setting="default",
    donor_random=False,
    ga_mode="topk",
    content="",
    write_step=10,
    sel_mode="wheel"
)

# Inicialización del evaluador
evaluator = SimEvaluator(args)

[2024-07-25 20:15:20,653] - ==================================================
[2024-07-25 20:15:20,655] - dev data: None
[2024-07-25 20:15:20,656] - test data: None
[2024-07-25 20:15:20,658] - 
	dataset = asset
	task = sim
	test_file = None
	batch_size = 20
	max_new_tokens = 128
	prompt_num = 0
	dev_file = None
	output = outputs/sim/asset/gpt/all/ga/bd2_top2_para_topk_init/topk/turbo/seed5
	language_model = gpt
	position = pre
	sample_num = 3
	seed = 5
	budget = 2
	popsize = 2
	evo_mode = ga
	llm_type = turbo
	initial = all
	initial_mode = para_topk
	para_mode = None
	ckpt_pop = None
	template = v1
	pred_mode = logits
	client = False
	cache_path = data/sim/asset/seed5/prompts_gpt.json
	setting = default
	donor_random = False
	ga_mode = topk
	content = 
	write_step = 10
	sel_mode = wheel
[2024-07-25 20:15:20,659] - ==================================================
[2024-07-25 20:15:20,660] - Instruction placeholder: <prompt>


Setting up log for basic
2000
10
359
10
[1275, 523, 1519, 734, 1628, 1414, 1930, 1723, 1515, 1335, 1888, 1085, 59, 1721, 953, 1589, 1931, 510, 1329, 106, 1845, 321, 231, 761, 960, 1778, 504, 779, 1113, 208, 1175, 26, 1497, 443, 835, 572, 372, 1876, 1776, 1568, 797, 326, 1560, 1632, 147, 284, 1265, 1264, 911, 259, 270, 3, 1991, 1784, 10, 428, 1584, 441, 1966, 339, 1786, 340, 592, 642, 1969, 407, 1104, 1794, 1388, 1281, 419, 1927, 1983, 403, 1979, 1976, 1830, 784, 611, 44, 739, 849, 1916, 298, 540, 133, 679, 617, 1674, 1235, 1200, 6, 1220, 1449, 692, 135, 634, 727, 1676, 626]
['Note that two storms on record, Hurricane Alice from the 1954 season and Tropical Storm Zeta from the 2005 season have formed during December and lasted into January.', "Blade of Fire is the second novel in Stuart Hill's fantasy series, the Icemark Chronicles.", 'It involved the WCW wrestlers "invading" WWF television in an attempt to "take over" the WWF.', 'He was the son of the Reverend Martin Luther King, Sr. a

In [13]:
# Ejemplo de uso
arg = argparse.Namespace(
    dataset="asset",
    dev_src=[
        "En un día soleado, María decidió ir a caminar por el parque con su perro, disfrutando del buen clima y de la tranquilidad del entorno.",
        "El presidente anunció en una conferencia de prensa que el nuevo programa económico traerá beneficios significativos a la población.",
        "El tren de alta velocidad reduce significativamente el tiempo de viaje entre las principales ciudades del país, permitiendo a los pasajeros llegar a su destino más rápido."
    ],
    dev_tgt=[
        ["María fue a caminar al parque con su perro en un día soleado."],
        ["El presidente dijo que el nuevo programa económico ayudará a la población."],
        ["El tren rápido reduce el tiempo de viaje entre ciudades."]
    ]
)

In [14]:
results = evaluator.forward(prompt_pre="Simplifica los siguientes prompts", eval_src=arg.dev_src, ref_texts=arg.dev_tgt)

print(f"SARI Score: {results['scores'][0]}")
for i, hypo in enumerate(results['hypos']):
    print(f"Original: {args.dev_src[i]}")
    print(f"Simplificación: {hypo}")
    print(f"Referencia: {args.dev_tgt[i]}\n")

[2024-07-25 20:15:23,350] - ### dataset example: En un día soleado, María decidió ir a caminar por el parque con su perro, disfrutando del buen clima y de la tranquilidad del entorno.



['En un día soleado, María decidió ir a caminar por el parque con su perro, disfrutando del buen clima y de la tranquilidad del entorno.', 'El presidente anunció en una conferencia de prensa que el nuevo programa económico traerá beneficios significativos a la población.', 'El tren de alta velocidad reduce significativamente el tiempo de viaje entre las principales ciudades del país, permitiendo a los pasajeros llegar a su destino más rápido.']
[['María fue a caminar al parque con su perro en un día soleado.'], ['El presidente dijo que el nuevo programa económico ayudará a la población.'], ['El tren rápido reduce el tiempo de viaje entre ciudades.']]


  0%|          | 0/3 [00:00<?, ?it/s]

turbo


 33%|███▎      | 1/3 [00:03<00:06,  3.49s/it]

turbo


 67%|██████▋   | 2/3 [00:06<00:03,  3.22s/it]

turbo


100%|██████████| 3/3 [00:08<00:00,  2.69s/it]


AssertionError: Reference sentences don't have the shape (n_references, n_samples)

## Clase Principal `Evoluter`

La clase `Evoluter` define la estructura básica y común para la evolución de prompts. Esta clase incluye métodos para inicializar la población de prompts, evaluar los prompts, ordenar los prompts según su desempeño y escribir los resultados en archivos de texto.

### Métodos Principales

- **`__init__(self, args, evaluator)`**: Constructor de la clase que inicializa las variables necesarias para la evolución de prompts.
  - **`args`**: Argumentos de configuración.
  - **`evaluator`**: Evaluador utilizado para calificar los prompts.

- **`sorted(self)`**: Ordena los prompts evaluados y escribe los resultados en un archivo de texto.
  - **`self.scores`**: Lista de puntuaciones de los prompts.
  - **`self.population`**: Lista de prompts en la población.
  - **`self.marks`**: Lista de marcas de los prompts.

- **`init_pop(self)`**: Inicializa la población de prompts, cargando prompts manuales y automáticos y evaluándolos.
  - **`manual_prompt_path`**: Ruta del archivo de prompts manuales.
  - **`ape_prompt_path`**: Ruta del archivo de prompts automáticos.
  - **`cache_path`**: Ruta del archivo de caché para guardar los prompts evaluados.

- **`write_step(self, step, best_score, avg_score)`**: Escribe los resultados de un paso de evolución en un archivo de texto.
  - **`step`**: Número de paso de la evolución.
  - **`best_score`**: Mejor puntuación del paso.
  - **`avg_score`**: Puntuación promedio del paso.

- **`evolute(self)`**: Método abstracto que debe ser implementado por las subclases. Define el proceso de evolución específico del algoritmo.

In [11]:
class Evoluter:
    def __init__(self, args, evaluator):
        self.evaluator = evaluator
        self.init_poplulation = []
        self.population = []
        self.scores = []
        self.marks = []
        self.client, self.llm_config = evaluator.client, evaluator.llm_config
        self.public_out_path = self.evaluator.public_out_path

        logger = self.logger = evaluator.logger
        logger.info("=" * 50)
        logger.info("\n\t" + "\n\t".join(f"{k} = {v}" for k, v in vars(args).items()))
        logger.info("=" * 50)
        self.args = args

        if args.task in ["sim", "sum"]:
            self.eval_src, self.eval_tgt = evaluator.dev_src, evaluator.dev_tgt
            self.eval_src = self.eval_src[: args.sample_num]
            self.eval_tgt = [i[: args.sample_num] for i in self.eval_tgt]
        elif args.task == "qa":
            self.eval_src, self.eval_tgt = evaluator.dev_src, evaluator.dev_tgt
        else:
            self.eval_src, self.eval_tgt = load_cls_data(
                evaluator.verbalizers, args.dev_file
            )

    def sorted(self):
        best_score = 0
        total_score = 0
        with open(os.path.join(self.public_out_path, "dev_result.txt"), "w") as wf:
            self.scores, self.population, self.marks = (
                list(t)
                for t in zip(
                    *sorted(
                        zip(self.scores, self.population, self.marks),
                        key=lambda x: x[0],
                        reverse=True,
                    )
                )
            )
            for score, prompt, mark in zip(self.scores, self.population, self.marks):
                score_str = "\t".join([str(round(i, 4)) for i in score])
                float_score = float(score[-1])
                if float_score > best_score:
                    best_score = float_score
                total_score += float_score
                wf.write(f"{mark}\t{prompt}\t{score_str}\n")
            wf.write(f"best score: {best_score}\n")
            wf.write(f"average score: {total_score / len(self.scores)}\n")
            wf.close()

    def init_pop(self):
        args = self.args
        evaluator = self.evaluator
        dataset = args.dataset
        prompts2mark = {}
        manual_prompt_path = f"./data/{args.task}/{dataset}/prompts.txt"
        ape_prompt_path = f"./data/{args.task}/{dataset}/prompts_auto.txt"
        if "gpt" in args.language_model or "opt" in args.language_model:
            model = f"_{args.language_model}"
        else:
            model = ""

        manual_pop = read_lines(manual_prompt_path)
        try:
            ape_pop = read_lines(ape_prompt_path)
        except:
            ape_pop = []
        for p in ape_pop:
            prompts2mark[p] = "ape"
        for p in manual_pop:
            prompts2mark[p] = "manual"

        self.evaluated_prompts = {}
        logger = self.logger
        out_path = self.public_out_path
        cur_budget = -1
        if args.initial == "all":
            cache_path = (
                args.cache_path
                if args.cache_path
                else f"./data/{args.task}/{dataset}/seed{args.seed}/prompts{model}.json"
            )
            try:
                self.evaluated_prompts = json.load(open(cache_path, "r"))
                logger.info(f"---loading prompts from {cache_path}")
                metric_index = -1
                self.evaluated_prompts = dict(
                    sorted(
                        self.evaluated_prompts.items(),
                        key=lambda item: item[1][metric_index],
                        reverse=True,
                    )
                )
                init_population = [k for k in list(self.evaluated_prompts.keys())]
            except:
                topk_population = []
                logger.info(
                    "-----evaluating initial population and paraphrasing topk---------"
                )
                for prompt in manual_pop + ape_pop:
                    eval_res = evaluator.forward(prompt, self.eval_src, self.eval_tgt)
                    scores = eval_res["scores"]
                    self.evaluated_prompts[prompt] = scores
                    topk_population.append((scores[-1], prompt))
                topk_population.sort(reverse=True, key=lambda x: x[0])

                with open(cache_path, "w") as wf:
                    self.evaluated_prompts = dict(
                        sorted(
                            self.evaluated_prompts.items(), key=lambda item: item[1][0]
                        )
                    )
                    json.dump(self.evaluated_prompts, wf)
                init_population = [i[1] for i in topk_population]
        elif args.initial == "ape":
            init_population = read_lines(ape_prompt_path)[: args.popsize]
            prompts2mark = {i: "ape" for i in init_population}
        elif args.initial == "ckpt":
            init_population = []
            logger.info(f"------------load from file {args.ckpt_pop}------------")
            ckpt_pop = read_lines(args.ckpt_pop)[: args.popsize]
            for line in ckpt_pop:
                try:
                    elements = line.split("\t")
                    mark, prompt = elements[0], elements[1]
                    score = elements[2:]
                    score = [float(i) for i in score]
                except:
                    continue
                prompts2mark[prompt] = mark
                self.evaluated_prompts[prompt] = [i for i in score]
                init_population.append(prompt)
            # print(init_population)
            cur_budget = extract_numbers(args.ckpt_pop.split("/")[-1])
            logger.info("cur budget is {}".format(cur_budget))

        client = evaluator.client
        llm_config = evaluator.llm_config

        # test LLM
        _ = paraphrase(
            sentence="Hi, I am a student.",
            type=args.llm_type,
            client=client,
            temperature=0.5,
            **llm_config,
        )
        logger.info("test LLM client success")
        if args.initial_mode in ["para_topk", "para_bottomk", "para_randomk"]:
            k_pop = k_init_pop(args.initial_mode, init_population, k=args.popsize)
            logger.info("-----paraphrasing topk---------")
            para_population = paraphrase(
                client=client, sentence=k_pop, type=args.llm_type, **llm_config
            )
            for p in para_population:
                prompts2mark[p] = "para"
                score = evaluator.forward(p, self.eval_src, self.eval_tgt)["scores"]
                self.evaluated_prompts[p] = score
            init_population = k_pop + para_population
            print(init_population)
            init_population = init_population[: args.popsize]
        elif args.initial_mode in ["topk", "bottomk", "randomk"]:
            init_population = k_init_pop(
                args.initial_mode, init_population, k=args.popsize
            )

        self.population = [i for i in init_population]
        assert len(self.population) == args.popsize

        for i in self.population:
            logger.info(i)
        with open(f"{out_path}/step0_pop_para.txt", "w") as wf:
            for prompt in self.population:
                score_str = "\t".join(
                    [str(round(i, 4)) for i in self.evaluated_prompts[prompt]]
                )
                wf.write(f"{prompts2mark[prompt]}\t{prompt}\t{score_str}\n")

        self.prompts2mark = prompts2mark
        return self.evaluated_prompts, cur_budget

    def write_step(self, step, best_score, avg_score):
        with open(os.path.join(self.public_out_path, f"step{step}_pop.txt"), "w") as wf:
            for p in self.population:
                score_str = "\t".join(
                    [str(round(i, 4)) for i in self.evaluated_prompts[p]]
                )
                wf.write(self.prompts2mark[p] + "\t" + p + "\t" + score_str + "\n")
            wf.write(f"best score: {best_score}\n")
            wf.write(f"average score: {avg_score}\n")

    def evolute(self):
        raise NotImplementedError

## Clase Heredada `GAEvoluter`

La clase `GAEvoluter` hereda de `Evoluter` y especializa el comportamiento para usar algoritmos genéticos en la evolución de prompts. Implementa el método `evolute` que define cómo se lleva a cabo la evolución usando la selección, mutación y cruce de prompts.

### Métodos Principales

- **`__init__(self, args, evaluator)`**: Constructor de la clase que inicializa las variables necesarias y carga la plantilla específica para la tarea.
  - **`args`**: Argumentos de configuración.
  - **`evaluator`**: Evaluador utilizado para calificar los prompts.

- **`evolute(self)`**: Implementa el algoritmo genético para la evolución de prompts.
  - **Inicialización**: Carga la población inicial usando `init_pop`.
  - **Ciclo de Evolución**: Realiza la selección de padres, generación de hijos y evaluación de prompts en cada paso del presupuesto (`budget`).
  - **Evaluación y Selección**: Actualiza la población con los mejores prompts según la estrategia especificada (`std`, `topk`).
  - **Escritura de Resultados**: Escribe los resultados de cada paso en un archivo de texto.

### Proceso de Evolución

1. **Inicialización**:
   - Carga la población inicial usando `init_pop`.
   - Configura variables y obtiene la mejor puntuación actual.

2. **Ciclo de Evolución**:
   - Para cada paso del presupuesto (`budget`), realiza las siguientes operaciones:
     - **Selección de Padres**: Usando la estrategia especificada (`wheel`, `random`, `tour`).
     - **Generación de Hijos**: Genera nuevos prompts (hijos) a partir de los padres seleccionados.
     - **Evaluación de Hijos**: Evalúa los prompts hijos y actualiza la población con los mejores.

3. **Evaluación y Selección**:
   - Evalúa cada nuevo prompt generado.
   - Actualiza la población con los mejores prompts según la estrategia especificada (`std`, `topk`).

4. **Escritura de Resultados**:
   - Escribe los resultados de cada paso en un archivo de texto.

In [6]:
class GAEvoluter(Evoluter):
    def __init__(self, args, evaluator):
        super(GAEvoluter, self).__init__(args, evaluator)
        try:
            self.template = templates_2[args.task]
        except:
            self.template = templates_2["sim"]

    def evolute(self):
        logger = self.logger
        self.evaluated_prompts, cur_budget = self.init_pop()
        evaluator = self.evaluator
        args = self.args
        eval_src = self.eval_src
        eval_tgt = self.eval_tgt
        out_path = self.public_out_path
        template = self.template

        best_scores = []
        avg_scores = []

        cur_best_prompt, cur_best_score = max(
            self.evaluated_prompts.items(), key=lambda x: x[1][0]
        )
        cur_best_score = cur_best_score[-1]
        fitness = np.array([self.evaluated_prompts[i][0] for i in self.population])

        for step in range(cur_budget + 1, args.budget):
            total_score = 0
            best_score = 0
            fitness = np.array([self.evaluated_prompts[i][0] for i in self.population])
            new_pop = []
            if args.sel_mode == "wheel":
                wheel_idx = np.random.choice(
                    np.arange(args.popsize),
                    size=args.popsize,
                    replace=True,
                    p=fitness / fitness.sum(),
                ).tolist()  # a temp pop to select parents
                parent_pop = [self.population[i] for i in wheel_idx]
            elif args.sel_mode in ["random", "tour"]:
                parent_pop = [i for i in self.population]

            for j in range(args.popsize):
                logger.info(f"step {step}, pop {j}")
                # print(np.random.choice(np.arange(args.popsize), size=2, replace=True,
                # p=fitness/fitness.sum()).tolist())
                if args.sel_mode in ["random", "wheel"]:
                    parents = random.sample(parent_pop, 2)
                    cand_a = parents[0]
                    cand_b = parents[1]
                elif args.sel_mode == "tour":
                    group_a = random.sample(parent_pop, 2)
                    group_b = random.sample(parent_pop, 2)
                    cand_a = max(group_a, key=lambda x: self.evaluated_prompts[x][0])
                    cand_b = max(group_b, key=lambda x: self.evaluated_prompts[x][0])

                request_content = template.replace("<prompt1>", cand_a).replace(
                    "<prompt2>", cand_b
                )
                # logger.info(f"old_child: {old_prompt}, {old_score}")
                logger.info("evolution example:")
                logger.info(request_content)
                logger.info("parents:")
                logger.info(cand_a)
                logger.info(cand_b)
                child_prompt = llm_query(
                    client=self.client,
                    data=request_content,
                    type=args.llm_type,
                    task=False,
                    temperature=0.5,
                    **self.llm_config,
                )
                logger.info(f"original child prompt: {child_prompt}")
                child_prompt = get_final_prompt(child_prompt)
                logger.info(f"child prompt: {child_prompt}")

                de_eval_res = evaluator.forward(child_prompt, eval_src, eval_tgt)
                de_hypos = de_eval_res["hypos"]
                de_scores = de_eval_res["scores"]
                de_score_str = "\t".join([str(round(i, 4)) for i in de_scores])
                new_score = de_scores[-1]

                logger.info(f"new score: {de_score_str}")
                self.prompts2mark[child_prompt] = "evoluted"

                self.evaluated_prompts[child_prompt] = de_scores
                if args.ga_mode == "std":
                    selected_prompt = child_prompt
                    selected_score = new_score
                    self.population[j] = selected_prompt

                elif args.ga_mode == "topk":
                    selected_prompt = child_prompt
                    selected_score = new_score

                new_pop.append(selected_prompt)
                total_score += selected_score
                if selected_score > best_score:
                    best_score = selected_score
                    if best_score > cur_best_score:
                        cur_best_score = best_score

            # self.population = new_pop
            if args.ga_mode == "topk":
                double_pop = list(set(self.population + new_pop))
                double_pop = sorted(
                    double_pop,
                    key=lambda x: self.evaluated_prompts[x][-1],
                    reverse=True,
                )
                self.population = double_pop[: args.popsize]
                total_score = sum(
                    [self.evaluated_prompts[i][-1] for i in self.population]
                )
                best_score = self.evaluated_prompts[self.population[0]][-1]
            avg_score = total_score / args.popsize
            avg_scores.append(avg_score)
            best_scores.append(best_score)

            self.write_step(step, best_score, avg_score)

            if step == args.budget - 1:
                logger.info(f"----------testing step {step} self.population----------")
                pop_marks = [self.prompts2mark[i] for i in self.population]
                pop_scores = [self.evaluated_prompts[i] for i in self.population]
                self.population, pop_scores, pop_marks = (
                    list(t)
                    for t in zip(
                        *sorted(
                            zip(self.population, pop_scores, pop_marks),
                            key=lambda x: x[1][-1],
                            reverse=True,
                        )
                    )
                )

                test_prompt_num = 3
                best_score, best_prompt = evaluate_optimized_prompt(
                    self.population[:test_prompt_num],
                    pop_marks[:test_prompt_num],
                    os.path.join(out_path, f"step{step}_pop_test.txt"),
                    evaluator,
                    args,
                )
                logger.info(
                    f"----------step {step} best score: {best_score}, best prompt: {best_prompt}----------"
                )

        best_scores = [str(i) for i in best_scores]
        avg_scores = [str(round(i, 4)) for i in avg_scores]
        logger.info(f"best_scores: {','.join(best_scores)}")
        logger.info(f"avg_scores: {','.join(avg_scores)}")
        self.scores = [self.evaluated_prompts[i] for i in self.population]
        self.marks = [self.prompts2mark[i] for i in self.population]
        self.sorted()


In [21]:
args = argparse.Namespace(
    dataset="asset",
    task="sim",
    test_file=None,
    batch_size=20,
    max_new_tokens=128,
    prompt_num=0,
    dev_file="./data/sim/asset/dev_100.txt",
    output="outputs/sim/asset/gpt/all/ga/bd2_top2_para_topk_init/topk/turbo/seed5",
    language_model="gpt",
    position="pre",
    sample_num=3,
    seed=5,
    budget=2,
    popsize=2,
    evo_mode="ga",
    llm_type="turbo",
    initial="all",
    initial_mode="para_topk",
    para_mode=None,
    ckpt_pop=None,
    template="v1",
    pred_mode="logits",
    client=False,
    cache_path="data/sim/asset/seed5/prompts_gpt.json",
    setting="default",
    donor_random=False,
    ga_mode="topk",
    content="",
    write_step=10,
    sel_mode="wheel"
)

# Inicialización del evaluador
evaluator = SimEvaluator(args)


[2024-07-25 20:09:03,249] - ==================================================
[2024-07-25 20:09:03,250] - dev data: ./data/sim/asset/dev_100.txt
[2024-07-25 20:09:03,251] - test data: None
[2024-07-25 20:09:03,252] - 
	dataset = asset
	task = sim
	test_file = None
	batch_size = 20
	max_new_tokens = 128
	prompt_num = 0
	dev_file = ./data/sim/asset/dev_100.txt
	output = outputs/sim/asset/gpt/all/ga/bd2_top2_para_topk_init/topk/turbo/seed5
	language_model = gpt
	position = pre
	sample_num = 3
	seed = 5
	budget = 2
	popsize = 2
	evo_mode = ga
	llm_type = turbo
	initial = all
	initial_mode = para_topk
	para_mode = None
	ckpt_pop = None
	template = v1
	pred_mode = logits
	client = False
	cache_path = data/sim/asset/seed5/prompts_gpt.json
	setting = default
	donor_random = False
	ga_mode = topk
	content = 
	write_step = 10
	sel_mode = wheel
[2024-07-25 20:09:03,252] - ==================================================
[2024-07-25 20:09:03,253] - Instruction placeholder: <prompt>


: 

In [15]:
# Ejemplo de uso
arg = argparse.Namespace(
    dataset="asset",
    dev_src=[
        "En un día soleado, María decidió ir a caminar por el parque con su perro, disfrutando del buen clima y de la tranquilidad del entorno.",
        "El presidente anunció en una conferencia de prensa que el nuevo programa económico traerá beneficios significativos a la población.",
        "El tren de alta velocidad reduce significativamente el tiempo de viaje entre las principales ciudades del país, permitiendo a los pasajeros llegar a su destino más rápido."
    ],
    dev_tgt=[
        ["María fue a caminar al parque con su perro en un día soleado."],
        ["El presidente dijo que el nuevo programa económico ayudará a la población."],
        ["El tren rápido reduce el tiempo de viaje entre ciudades."]
    ]
)

In [16]:
results = evaluator.forward(prompt_pre="Simplifica los siguientes prompts", eval_src=arg.dev_src, ref_texts=arg.dev_tgt)

print(f"SARI Score: {results['scores'][0]}")
for i, hypo in enumerate(results['hypos']):
    print(f"Original: {args.dev_src[i]}")
    print(f"Simplificación: {hypo}")
    print(f"Referencia: {args.dev_tgt[i]}\n")

[2024-07-25 20:06:35,782] - ### dataset example: En un día soleado, María decidió ir a caminar por el parque con su perro, disfrutando del buen clima y de la tranquilidad del entorno.

  0%|          | 0/3 [00:00<?, ?it/s]

turbo


 33%|███▎      | 1/3 [00:03<00:07,  3.90s/it]

turbo


 67%|██████▋   | 2/3 [00:06<00:03,  3.21s/it]

turbo


100%|██████████| 3/3 [00:09<00:00,  3.32s/it]


AssertionError: Reference sentences don't have the shape (n_references, n_samples)